# [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-2.2-0075ff)](https://stablediffusion.vn/update/) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
🧨 `UI Colab - A100`

In [ ]:
#@title ✨ Cài đặt UI
#@markdown `Chạy cài đặt để kết nối drive và tải dữ liệu nền`
%cd /content
from google.colab import drive
drive.mount('/content/drive')

UI = 'Colab'

root_dir = "/content"
repo_dir = f"{root_dir}/SDVN-training-colab-flux"
toolkit_dir = f"{repo_dir}/ai-toolkit"
config_dir = f"{repo_dir}/Config"

%cd {root_dir}
import os
if not os.path.exists(repo_dir):
  !git clone  --recurse-submodules https://github.com/StableDiffusionVN/SDVN-training-colab-flux {repo_dir}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={repo_dir}/libtcmalloc_minimal.so.4
%cd {toolkit_dir}
!pip install -q -r requirements.txt

%run {repo_dir}/TrainScript.ipynb

if UI == "Gradio":
  !rm -r {toolkit_dir}/output
  !ln -s /content/drive/MyDrive/SD-Data/Lora {toolkit_dir}/output
  !python {toolkit_dir}/flux_train_ui.py


In [ ]:
#@title ✨ 2. Train

#@markdown `Trigger word sẽ tự động đặt theo tên lora. Lora sẽ tự động lưu tại Drive/SD-Data/Lora`
No_gen_caption = False #@param {type:"boolean"}
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False
Caption = 'Florence'
Caption_Length = "Medium"
Joy_Type = "Descriptive"
Joy_Extra_Option = ""
Joy_Character_Name = ""
Joy_Custom_Prompt = ""
Lora_name = "Lora name - Trigger Word" # @param {type:'string'}
Custom_Caption = Lora_name
AddFolderName = True
Remove_Caption = False
Append = False
extension =".txt"
train_folder_list = TrainFolder.split(",")

if No_gen_caption == True:
  Caption = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',10,30,'short'],
    'Medium':['<DETAILED_CAPTION>',10,100,'medium-length'],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,'very long']
}

for folder in train_folder_list:
  if DataClean == True :
    %cd /content
    clean_directory(folder)
  if Caption != "None":
    caption_dir(folder,Cap_prompt[Caption_Length][0])
  if AddFolderName:
    add_forder_name(folder)
  if Custom_Caption != "":
    process_dir(folder, Custom_Caption, Append, Remove_Caption)

Low_VRAM = False
OutputFolder = "/content/drive/MyDrive/SD-Data/Lora"
#@markdown `Số lần repeats sẽ quyết định độ mạnh yếu của lora (nên từ 200-400 với số ảnh < 50)`
Repeats = 200 # @param {type:'number'}
Save_steps = 500
Batch_size = 2
Lr = 1e-4
Dim = 32
Alpha = 16
Resolution = "[1024]"
folder_train = []
for folder in train_folder_list:
  if len(get_supported_images(folder)) > 0:
    Steps = len(get_supported_images(folder)) * Repeats
    folder_dic = {
    "path": folder,
    "steps": Steps,
    "name": get_steps(folder)[1].replace(" ", "_")
    }
    folder_train.append(folder_dic)
  else:
    print(f"Thư mục [ {folder} ] có thể không chứa ảnh được hỗ trợ, hãy kiểm tra lại (.png, .jpg, .jpeg, .webp, .bmp, .JPG, .JPEG, .PNG)")

for folder in folder_train:
  file_path = f'{config_dir}/config_{folder["name"]}.yaml'
  !cp "{config_dir}/config.yaml" "{file_path}"
  if AddFolderName:
    process_dir(folder["path"], folder["name"], False, False)
  config = {
      'data': [r'folder_path:.*',f'folder_path: "{folder["path"]}"'],
      'name': [r'name:.*',f'name: "{Lora_name if len(folder_train) == 1 else folder["name"]}"'],
      'output': [r'training_folder:.*',f'training_folder: "{OutputFolder}"'],
      'steps': [r' steps:.*',f' steps: {folder["steps"]}'],
      'save_steps': [r'save_every:.*',f'save_every: {Save_steps}'],
      'sample_every': [r'sample_every:.*',f'sample_every: {Save_steps}'],
      'resolution': [r'resolution:.*',f'resolution: {Resolution}'],
      'batch': [r'batch_size:.*',f'batch_size: {Batch_size}'],
      'dim': [r'linear:.*',f'linear: {Dim}'],
      'alpha': [r'linear_alpha:.*',f'linear_alpha: {Alpha}'],
      'lr': [r'lr:.*',f'lr: {Lr}'],
      'prompt': [r'prompts:[\s\S]*?neg:',f'prompts:\n          - "{random_sample(folder["path"])}"\n        neg:'],
      'low_vram': [r'low_vram:.*',f'low_vram: {"true" if Low_VRAM == True else "false"}'],

  }
  for i in config:
    replace(config[i][0],config[i][1])

  print('=====================')
  print(f'Thư mục train: {folder["path"]}')
  print(f'  Số lượng ảnh: {len(get_supported_images(folder["path"]))}')
  print(f'  Số Steps: {folder["steps"]}')
  print(f'  Output: {OutputFolder}/{folder["name"]}')
  print('=====================')

AutoDisconect = False #@param {type:"boolean"}
train()
if AutoDisconect:
    from time import sleep
    sleep(180)
    from google.colab import runtime
    runtime.unassign()
